#### Файл сервера xmlrpc_stats_server_p3.ipynb на Python 3 

In [1]:
max_log_count = 15
log_archiving_enabled = True

In [ ]:

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler

import re
import os
import datetime

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8018), 
                            requestHandler=RequestHandler)


# Добавление строки в лог
def add_log(sname):
    current_log_count = get_log_lines_count()
    if (current_log_count >= max_log_count):
        archive_log_file()    
    f = open('log.csv','a')
    f.write(str(sname)+','+ datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +'\n')
    f.close()
    return True
server.register_function(add_log, 'add_log')

def get_log(start_time = '2000-01-01 00:00:00', end_time = '2100-01-01 00:00:00', operation_type = None):
    start_time = datetime.datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
    end_time = datetime.datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S')
    result_logs_lines = []
    logs_files = get_logs_files()
    for logs_file in logs_files:
        file = open(logs_file, 'r')
        lines = file.readlines()
        for line in lines:
            line_tokens = line.split(",")
            type = line_tokens[0]
            time = datetime.datetime.strptime(line_tokens[1].removesuffix("\n"), '%Y-%m-%d %H:%M:%S')
            if time >= start_time and time <= end_time and (operation_type is None or operation_type == type):
                result_logs_lines.append(line)
    return result_logs_lines
server.register_function(get_log, 'get_log')

def get_log_lines_count():
    f = open('log.csv','r')
    count = 0
    while (f.readline()):
        count += 1
    f.close()
    return count
    
def archive_log_file():
    old_file = open('log.csv','r')
    new_file_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_log.csv"
    new_file = open(new_file_name, 'x')
    new_file.writelines(old_file.readlines())
    new_file.close()
    old_file.close()
    old_file = open('log.csv','w')
    old_file.truncate(0)
    old_file.close()

def get_log(start_time, end_time, operation_type):
    result_logs_lines = []
    logs_files = get_logs_files()
    for logs_file in logs_files:
        file = open(logs_file, 'r')
        lines = file.readlines()
        for line in lines:
            line_tokens = line.split(",")
            type = line_tokens[0]
            time = line_tokens[1]
            print(type)
            print(time)

def get_logs_files():
    rootdir = "."
    regex = re.compile('(.*csv$)')
    result = []
    for root, dirs, files in os.walk(rootdir):
        for file in files:
            if regex.match(file):
                result.append(file)
    return result


print ("Listening on port 8018...")
server.serve_forever()


Listening on port 8018...


127.0.0.1 - - [09/Oct/2023 22:56:08] "POST /RPC2 HTTP/1.1" 200 -
